In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/olya-
[nltk_data]     const/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Define train/validation/test Datasets & get the vocabulary:

In [29]:
import pickle

from datasets.flickr8k import Flickr8kDataset


DATASET_BASE_PATH = 'data/flickr8k/'
VOCAB = 'vocab_set.pkl'

train_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='train',
    device=device, return_type='tensor', load_img_to_memory=False)

vocab_set = train_set.get_vocab()
with open(VOCAB, 'wb') as f:
    pickle.dump(vocab_set, f)
    

vocab, word2idx, idx2word, max_len = vocab_set
vocab_size = len(vocab)

val_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='val', vocab_set=vocab_set,
    device=device, return_type='corpus', load_img_to_memory=False)

test_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='test', vocab_set=vocab_set,
    device=device, return_type='corpus', load_img_to_memory=False)

train_eval_set = Flickr8kDataset(
    dataset_base_path=DATASET_BASE_PATH, dist='train', vocab_set=vocab_set,
    device=device, return_type='corpus', load_img_to_memory=False)


print(
    f"The number of samples in:\ntrain: {len(train_set)};"
    + f" validation: {len(val_set)}; test: {len(test_set)}\n"
    + f"Vocabulary size: {vocab_size}; Max length of a sentence: {max_len};"
)

The number of samples in:
train: 30000; validation: 1000; test: 1000
Vocabulary size: 7708; Max length of a sentence: 40;


# Define Data Transformations & Dataloaders:

In [30]:
from torchvision import transforms
from torch.utils.data import DataLoader


BATCH_SIZE = 16


train_transformations = transforms.Compose([
    transforms.Resize(256),  # smaller edge of image resized to 256
    transforms.RandomCrop(224),  # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),  # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),  # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))
])

eval_transformations = transforms.Compose([
    transforms.Resize(256),  # smaller edge of image resized to 256
    transforms.CenterCrop(224),  # get 224x224 crop from random location
    transforms.ToTensor(),  # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),  # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))
])

train_set.transformations = train_transformations
val_set.transformations = eval_transformations
test_set.transformations = eval_transformations
train_eval_set.transformations = eval_transformations


eval_collate_fn = lambda batch: (torch.stack([x[0] for x in batch]), [x[1] for x in batch], [x[2] for x in batch])
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, sampler=None, pin_memory=False)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                        collate_fn=eval_collate_fn)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                         collate_fn=eval_collate_fn)
train_eval_loader = DataLoader(train_eval_set, batch_size=BATCH_SIZE, shuffle=False, sampler=None, pin_memory=False,
                               collate_fn=eval_collate_fn)

# Define the model, loss, metric, optimizer, embedding_matrix:

#### Note: you can use:
    - densenet161_gru
    - densenet161_lstm

#### Probably you can add the other models as well and use the notebook, but check!

In [31]:
from models.torch.densenet161_lstm import Captioner

In [32]:
from glove import embedding_matrix_creator
from metrics import bleu_score_fn, accuracy_fn 
from utils_torch import words_from_tensors_fn

EMBEDDING_DIM = 50
EMBEDDING = f"GLV{EMBEDDING_DIM}"
HIDDEN_SIZE = 256
LR = 1e-2

embedding_matrix = embedding_matrix_creator(embedding_dim=EMBEDDING_DIM, word2idx=word2idx)
print(f"Embedding matrix shape: {embedding_matrix.shape}")

100%|██████████| 7708/7708 [00:00<00:00, 508912.67it/s]

Embedding matrix shape: (7708, 50)


In [33]:
final_model = Captioner(EMBEDDING_DIM, HIDDEN_SIZE, vocab_size, num_layers=2,
                        embedding_matrix=embedding_matrix, train_embd=False).to(device)
MODEL = final_model.name

In [34]:
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=train_set.pad_value).to(device)
acc_fn = accuracy_fn(ignore_value=train_set.pad_value)
sentence_bleu_score_fn = bleu_score_fn(4, 'sentence')
corpus_bleu_score_fn = bleu_score_fn(4, 'corpus')
tensor_to_word_fn = words_from_tensors_fn(idx2word=idx2word)

params = (
    list(final_model.decoder.parameters())
    + list(final_model.encoder.embed.parameters())
    + list(final_model.encoder.bn.parameters())
)

optimizer = torch.optim.Adam(params=params, lr=LR)

# Main training cycle:

**Note:** modify train_from_scratch=True if you want to get your model!

In [ ]:
train_from_scratch = False

from utils_torch import check_create_dir, get_picture_caption
from train_torch import train_model, evaluate_model
from PIL import Image

if train_from_scratch:
    check_create_dir(f'saved_models/{MODEL}')
    MODEL_NAME = f'saved_models/{MODEL}/{MODEL}_b{BATCH_SIZE}_emd{EMBEDDING}'
    NUM_EPOCHS = 100
    print(MODEL_NAME)


    train_loss_min = 100
    val_bleu4_max = 0.0

    for epoch in range(NUM_EPOCHS):
        train_loss = train_model(
            desc=f'Epoch {epoch + 1}/{NUM_EPOCHS}',
            model=final_model,                                                                      
            optimizer=optimizer,
            loss_fn=loss_fn,                                          
            train_loader=train_loader,
            acc_fn=acc_fn
        )

        with torch.no_grad():
            train_bleu = evaluate_model(
                desc=f'\tTrain Bleu Score: ',
                model=final_model,
                bleu_score_fn=corpus_bleu_score_fn,
                tensor_to_word_fn=tensor_to_word_fn,
                data_loader=train_eval_loader
            )

            val_bleu = evaluate_model(
                desc=f'\tValidation Bleu Score: ',
                model=final_model,
                bleu_score_fn=corpus_bleu_score_fn,
                tensor_to_word_fn=tensor_to_word_fn,
                data_loader=val_loader
            )

            print(f'Epoch {epoch + 1}/{NUM_EPOCHS}',
                ''.join([f'train_bleu{i}: {train_bleu[i]:.4f} ' for i in (1, 4)]),
                ''.join([f'val_bleu{i}: {val_bleu[i]:.4f} ' for i in (1, 4)]),
                )
            
            state = {
                'epoch': epoch + 1,
                'state_dict': final_model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'train_loss_latest': train_loss,
                'val_bleu4_latest': val_bleu[4],
                'train_loss_min': min(train_loss, train_loss_min),
                'val_bleu4_max': max(val_bleu[4], val_bleu4_max),
                'train_bleus': train_bleu,
                'val_bleus': val_bleu,
            }

            torch.save(state, f'{MODEL_NAME}_latest.pt')

            if train_loss < train_loss_min:
                train_loss_min = train_loss
                torch.save(state, f'{MODEL_NAME}''_best_train.pt')
                
            if val_bleu[4] > val_bleu4_max:
                val_bleu4_max = val_bleu[4]
                torch.save(state, f'{MODEL_NAME}''_best_val.pt')

    torch.save(state, f'{MODEL_NAME}_ep{NUM_EPOCHS:02d}_weights.pt')

# Get best checkpoint results:

In [ ]:
#TODO